### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Sample from web page
# https://api.nytimes.com/svc/search/v2/articlesearch.json?q=new+york+times&page=2&sort=oldest&api-key=your-api-key 
#  Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url=(f'{url}fq={filter_query}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&sort={sort}')
print (query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&begin_date=20130101&end_date=20230531&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&sort=newest


In [4]:
# Create an empty list to store the reviews
love_movie_review = list()


# loop through pages 0-19
for page in range(0,10):
    # create query with a page number
    pages = (f'{query_url}&page={str(page)}&api-key={nyt_api_key}')

    # API results show 10 articles at a time
    
    # Make a "GET" request and retrieve the JSON
    request = requests.get(pages).json()
    print (f'{page} {request}')
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep (12)
    
    # Try and save the reviews to the reviews_list
    if not request["response"]["docs"]:
        # Print the page number that had no results then break from the loop
        print (f'{page} had to results')
        break 
        # loop through the reviews["response"]["docs"] and append each review to the list
    else:
        for article in request["response"]["docs"]:
        # Print the page that was just retrived
            love_movie_review.append(article)

0 {'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}], 'pub_date': '2023-05-25T11:00:03+0000', 'byline': {'original': 'By Jeannette Catsoulis', 'person': [{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'q

In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print (json.dumps(love_movie_review[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
love_movie_review_df=pd.json_normalize(love_movie_review)
love_movie_review_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [7]:
# Extract the title from the "headline.main" column and
# Regular expression to match text enclosed by ‘ and ’
pattern = r"(?:\u0020|^)\u2018(.+?)\u2019(?:\u003A|\u0020|$)"
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# # Use re.search to find a match to the pattern
for idx, row in love_movie_review_df.iterrows():
    title = re.search(pattern, row['headline.main'])
    if title:
        love_movie_review_df.loc[idx,'title']=title.group(1)
        #print (f"{idx}  {love_movie_review_df.loc[idx,'headline.main']}\t\t Title: {love_movie_review_df.loc[idx,'title']}")
    else:
        print (f'{idx}  Title not found')
        print (f"{idx}  {love_movie_review_df.loc[idx,'headline.main']}\t\t Title: {love_movie_review_df.loc[idx,'title']}")
        love_movie_review_df.loc[idx,'title'] = "not found"

# -----End string should include " Review" to avoid cutting title early----
# "" Revie"w doesnt work "Review: ‘What’s Love Got to Do With It?’ Proba..." not all titles come first. opted for u0020 before and after. 	


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string

for idx, row in love_movie_review_df.iterrows():
    love_movie_review_df.loc[idx,'keywords']=extract_keywords(row['keywords'])
    print(f"{idx}   {love_movie_review_df.loc[idx,'keywords']}")
    # test =extract_keywords(row['keywords'])
    # print (love_movie_review_df.head(1).loc[idx,'keywords'])



0   subject: Movies;creative_works: The Attachment Diaries (Movie);persons: Diment, Valentin Javier;
1   subject: Movies;persons: Kapur, Shekhar;persons: James, Lily;persons: Azmi, Shabana;persons: Thompson, Emma;
2   subject: Movies;creative_works: You Can Live Forever (Movie);persons: Slutsky, Mark;persons: Watts, Sarah (Film Director);persons: O'Driscoll, Anwen;persons: Laporte, June (Actor);
3   subject: Movies;creative_works: A Tourist's Guide to Love (Movie);persons: Tsuchida, Steven;persons: Cook, Rachael Leigh;
4   subject: Movies;persons: Zlotowski, Rebecca;creative_works: Other People's Children (Movie);
5   subject: Movies;persons: Bracey, Luke (1989- );persons: Liu, Simu;persons: Reid, Taylor Jenkins;persons: Soo, Phillipa;persons: Fickman, Andy;creative_works: One True Loves (Movie);
6   subject: Documentary Films and Programs;creative_works: The Lost Weekend: A Love Story (Movie);persons: Pang, May;persons: Lennon, John;persons: Ono, Yoko;persons: Brandstein, Eve;persons:

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = love_movie_review_df['title'].to_list()
titles_list


['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journa

### Access The Movie Database API

In [31]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
tmdb_key_string1 = "?api_key=" + tmdb_api_key

In [44]:
# Create an empty list to store the results
empty_list=[]

# Create a request counter to sleep the requests after a multiple
# of 50 requests
req_counter = 0

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if req_counter > 18:
        time.sleep(15)
        req_counter=0
    else:
    # Add 1 to the request counter
        req_counter += 1

    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id    
        movie_id=requests.get(f'https://api.themoviedb.org/3/search/movie?query={title}{tmdb_key_string}').json()['results'][0]['id']
        if response.status_code == 200:
            print(f"Movie ID Found: {movie_id}")            
        # Make a request for a the full movie details
            movie_detail_df=requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}{tmdb_key_string1}").json()

        # Execute "GET" request with url

            print (movie_detail_df)
        
        # Extract the genre names into a list

        # Extract the spoken_languages' English name into a list

        # Extract the production_countries' name into a list

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found
            print (f'{req_counter}  {title}')
    
    
        else:
            print ("next")
    except requests.exceptions.RequestException as e:
    # This catches all exceptions that are requests-related
    # Including connection errors, timeouts, etc.
        print("A network error occurred. Please try again later.")
        print(e)


Movie ID Found: 743040
{'adult': False, 'backdrop_path': None, 'belongs_to_collection': None, 'budget': 0, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 9648, 'name': 'Mystery'}, {'id': 53, 'name': 'Thriller'}, {'id': 27, 'name': 'Horror'}], 'homepage': '', 'id': 743040, 'imdb_id': 'tt11359158', 'original_language': 'es', 'original_title': 'El apego', 'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.', 'popularity': 1.601, 'poster_path': '/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg', 'production_companies': [], 'production_countries': [{'iso_3166_1': 'AR', 'name': 'Argentina'}], 'release_date': '2021-10-07', 'revenue': 0, 'runtime': 102, 'spoken_langu

IndexError: list index out of range

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [ ]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
